# Merge all tokens with revisions

In [ ]:
import pandas as pd
from wikiwho_wrapper import WikiWho
from os.path import exists

filename = 'data/freddie_mercury.df'
try:
    df = pd.read_pickle(filename)
except:
    ww = WikiWho()
    df = ww.dv.all_content("Freddie Mercury")
    df = df[~((df['in'] == -1) & (df['out'] == -1))]
    df = df.drop(columns=['article_title', 'page_id']) 

    revisions = ww.dv.rev_ids_of_article('Freddie Mercury')

    df = pd.merge(df,
                  revisions.rename(
                      columns={
                          'rev_id': 'in',
                          'o_editor': 'in_editor',
                          'rev_time': 'in_rev_time'
                      })[['in', 'in_editor', 'in_rev_time']],
                  how='left', on='in')

    df = pd.merge(df,
                  revisions.rename(
                      columns={
                          'rev_id': 'out',
                          'o_editor': 'out_editor',
                          'rev_time': 'out_rev_time'
                      })[['out', 'out_editor', 'out_rev_time']],
                  how='left', on='out')


    df.to_pickle(filename)


# Query all revisions that reinserted a token

In [ ]:
ins = df.loc[(df['token'] == 'british') & (df['in'] != -1), ['token_id', 'in', 'in_rev_time', 'in_editor']]
ins.head()

# Query all revisions that removed a token

In [ ]:
outs = df.loc[(df['token'] == 'british') & (df['out'] != -1), ['token_id', 'out', 'out_rev_time', 'out_editor']]
outs.head()